In [17]:
# Funtkion um nur neue Daten herunterzuladen

In [18]:
# Module Imports

#DB stuff
import mariadb
import sys

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time
import datetime


In [19]:
#Connect with DB
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [20]:
# Verbindungsinformationen für den Air-q Sensor
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]

def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

def pad(data):
  length = 16 - (len(data) % 16)
  return data + chr(length).encode('utf-8')*length

def encodeMessage(msg):
  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Erster Schritt: AES256 verschlüsseln
  iv = Random.new().read(AES.block_size)
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
  msg = msg.encode('utf-8')
  crypt = iv + cipher.encrypt(pad(msg))

  # Zweiter Schritt: base64 enkodieren
  msgb64 = base64.b64encode(crypt).decode('utf-8')
  return msgb64

#Gibt ein JSON-Objekt über die verfügbaren Daten zurück
def getAvailable():
    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Daten anfordern
    connection.request("GET", "/dirbuff")
    contents = connection.getresponse()

    # Daten entschlüsseln
    msg = decodeMessage(contents.read())
  
    #JSON-String umwandeln in dict
    folder_dict = json.loads(msg)
    
    # Verbindung trennen
    connection.close()

    return folder_dict

In [21]:
# Funktion zur Überprüfung des aktuellen Zeitstempels in der lokalen DB
def getLatestTimestamp():
    sql = "SELECT MAX(timestamp) FROM measurements;"
    cur.execute(sql)
    result = cur.fetchone()
    if result[0] is not None:
        return int(time.mktime(result[0].timetuple()) * 1000)
    else:
        return 0

In [22]:
def sql_data(contents, cur):
    latest_timestamp = getLatestTimestamp()
    
    for line in contents.read().split(b'\n'):
        if line != b'':
            line = decodeMessage(line)
            print(line)
            line = json.loads(line)
            
            if line["Status"] != "OK":
                print(line["Status"])
                print("Skipped invalid measurements due to warm-up of the Sensor")
                continue

            # Überprüfen, ob der Timestamp größer als der neueste Timestamp in der Datenbank ist
            line_timestamp = line.get("timestamp", 0)
            latest_timestamp = getLatestTimestamp()

            if line_timestamp > latest_timestamp:
                columns = ""
                values = ""

                for key, value in line.items():
                    if key in ["bat", "DeviceID", "uptime", "window_event", "door_event", "person", "window_open", "Status"]:
                        continue
                    
                    columns += key

                    if isinstance(value, list):
                        values += str(value[0])
                    elif key == "timestamp":
                        values += "FROM_UNIXTIME(%s)" % (int(value / 1000))
                    else:
                        values += str(value)

                    if key != "cnt0_3":
                        columns += ", "
                        values += ", "

                #if result is None:
                sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s );" % ('measurements', columns, values)
                print(sql)
                cur.execute(sql)
                conn.commit()
                print("Messdaten eingefügt")
                return "Messdaten eingefügt"
            else:
                print("Messdaten bereits vorhanden")
                return "Messdaten bereits vorhanden"
                break



In [23]:
def file_to_db():
    folder = getAvailable()

    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)
    
    data_exits = False

    for year in folder.keys():
        for month in folder[year].keys():
            for day in folder[year][month].keys():
                for file in folder[year][month][day]:
                    # Anfrage formulieren und Daten anfordern
                    print("_____________________________________________________________________________")
                    print("Year: " + year + "\n" + "Month: " + month + "\n" + "Day: " + day + "\n" + "File: " + file + "\n")   
                    connection.request("GET","/file?request="+encodeMessage(year+"/"+month+"/"+day+"/"+file))
                    contents = connection.getresponse()
                    if contents.status == 200:
                        print("Status: " + str(contents.status) + "/OK")
                    else:
                        print("Status: " + str(contents.status) + "/Canceled!")
                        break
                    sql_data(contents, cur)
                    


    # Verbindung trennen
    connection.close()

In [24]:
file_to_db()

_____________________________________________________________________________
Year: 2023
Month: 5
Day: 30
File: 1685404738

Status: 200/OK
{"TypPS": 14.997, "oxygen": [20.925, 2.65], "bat": [0, 0], "pm10": [0.001, 10.0], "cnt0_5": [25.261, 12.59], "co": [1.519, 0.2], "temperature": [21.312, 0.54], "performance": 903, "uptime": 1735992, "window_event": 0, "co2": [444.621, 63.34], "measuretime": 1787, "DeviceID": "de7fd19cce1c108a714ab9063e3a7518", "so2": [148.949, 44.12], "no2": [41.96, 3.19], "cnt5": [0.001, 10.0], "timestamp": 1685404738000, "pm1": [0.0, 10.0], "door_event": 0, "cnt1": [0.909, 10.01], "dewpt": [8.312, 1.1], "Status": "OK", "tvoc": [690.712, 103.6], "pressure": [1000.495, 1.0], "cnt10": [0.0, 10.0], "dCO2dt": -0.37, "sound_max": 54.9, "health": 762, "temperature_o2": [25.409, 1.0], "cnt2_5": [0.004, 10.0], "o3": [15.495, 1.37], "humidity": [44.236, 3.73], "dHdt": -0.02, "person": 0, "window_open": 0, "humidity_abs": [8.285, 0.59], "sound": [53.72, 2.69], "pm2_5": [0.0,